In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic_2d
import seaborn as sns
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, LogTicker
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models.annotations import Label
from bokeh.models import CustomJS
import datetime as dtt
from matplotlib import cm

In [2]:
import cPickle as pickle
def save_obj(data, name):
    with open(name, 'wb') as fp:
        pickle.dump(data, fp)
        
def load_obj(name):
    with open(name, 'rb') as fp:
        return pickle.load(fp)
    
##Перевод координат в формат WebMercator
def wgs84_to_web_mercator(longitude, latitude):
    """Converts decimal longitude/la94728titude to Web Mercator format"""
    k = 6378137
    x = longitude * (k * np.pi/180.0)
    y = np.log(np.tan((90 + latitude) * np.pi/360.0)) * k
    return x, y

In [3]:
npickups    = pd.read_csv('npickups_june.csv',     parse_dates=True, index_col='datetime')
predictions = pd.read_csv('predictions_C6_W6.csv', parse_dates=True, index_col='datetime')
patch_X, patch_Y = load_obj('X_patches.p'), load_obj('Y_patches.p')

In [4]:
#Координаты границ зоны видимости на карте
NY_lat_0 = 40.49612
NY_lat_1 = 40.91553
NY_lon_0 = -73.70001
NY_lon_1 = -74.25559

#Координаты в формате web mercator
NY_coord_0_wmer = wgs84_to_web_mercator(NY_lon_0, NY_lat_0)
NY_coord_1_wmer = wgs84_to_web_mercator(NY_lon_1, NY_lat_1)

In [5]:
npickups.values.max()

2060

In [6]:
from bokeh.io import output_file, show, curdoc
from bokeh.models import CustomJS, ColorBar, LogTicker
from bokeh.layouts import widgetbox, row
from bokeh.models.widgets import Slider
from bokeh.palettes import Inferno256 as palette
color_mapper = LogColorMapper(palette=palette)


Для упрощения реализации отображения прогноза разобъем датафрейм с предсказаниями на 6 частей и передадим их в массив.

In [7]:
predictions_list = []
for dt in range(1, 7):
    names = []
    for col in npickups.columns:
        names.append('prediction_{}_dt_{}'.format(col, dt))
    predictions_list.append(predictions.loc[:, names])

In [8]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure, output_file, show

start_slider_day = 0
start_slider_time = 15
start_slider_forecast = 3
start_datetime = pd.to_datetime('2016-06-01 {}h'.format(start_slider_time))

source = ColumnDataSource(data=dict(
    x        = patch_X,
    y        = patch_Y,
    name     = npickups.columns,
    npickups = predictions_list[start_slider_forecast - 1].loc[start_datetime,:],
))

plot = figure(tools='pan, wheel_zoom, hover, reset', 
              title='{} of {}, 2016. Time: {}. A {} hours \
              NY taxi pickups forecast'.format(start_datetime.day,
                                               pd.datetime.strftime(start_datetime, '%B'),
                                               pd.datetime.strftime(start_datetime, '%I %p'),
                                               start_slider_forecast),
              x_range=(NY_coord_1_wmer[0], NY_coord_0_wmer[0]),
              y_range=(NY_coord_1_wmer[1], NY_coord_0_wmer[1]))

plot.patches('x', 'y', source=source,
          fill_color={'field': 'npickups', 'transform': color_mapper},
          fill_alpha=0.4, line_color="white", line_width=0.5)

def update_data(attr, old, new):
    day      = slider_day.value
    time     = slider_time.value
    forecast = slider_forecast.value
    tdate = pd.to_datetime('2016-06-{} {}h'.format(day, time))
    plot.title.text = '{} of {}, 2016. {}. A {} hours \
    NY taxi pickups forecast'.format(tdate.day,
                                     pd.datetime.strftime(tdate, '%B'),
                                     pd.datetime.strftime(tdate, '%I %p'),
                                     forecast)
    # Set new_data
    new_data = dict(x=patch_X, y=patch_Y, name=npickups.columns,
                    npickups=predictions_list[forecast - 1].loc[tdate,:])
    # Assign new_data to: source.data
    source.data = new_data

# Make a slider object: slider
slider_day      = Slider(title = 'day', start=1, end=31, step=1, value=start_slider_day)
slider_time     = Slider(title = 'time ', start=0, end=24, step=1, value=start_slider_time)
slider_forecast = Slider(title = 'forecast ', start=1, end=6, step=1, value=start_slider_forecast)

# Attach the callback to the 'value' property of slider
for w in [slider_day, slider_time, slider_forecast]:
    w.on_change('value', update_data)
    
hover = plot.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Pickups", "@npickups"),
    ("(Long, Lat)", "($x, $y)"),
]

plot.axis.visible = True
plot.add_tile(CARTODBPOSITRON)
color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

plot.add_layout(color_bar, 'right')

# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = column(widgetbox(slider_day, slider_time, slider_forecast), plot)
curdoc().add_root(layout)
show(layout)